In [1]:
from chan_modeling.archiving import MyThreads, clean
from chan_modeling.lda_modeling import train_lda_model, prepare_dictionary, get_topics_for_document
from chan_modeling.functional_helpers import nth
import gensim
from gensim.models.ldamodel import LdaModel
import pprint
from collections import Counter
from operator import itemgetter

Slow version of gensim.models.doc2vec is being used


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
threads = MyThreads('chan_modeling/biz_archive')

In [3]:
# phrases = gensim.models.Phrases(threads)

In [4]:
# bigrams = gensim.models.phrases.Phraser(phrases)

In [5]:
# nth(bigrams[threads], 45)

In [6]:
doc_term_matrix, dictionary = prepare_dictionary(threads)

2018-07-23 05:20:33,119 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-23 05:21:07,445 : INFO : adding document #10000 to Dictionary(93571 unique tokens: ['nuls', 'status', 'update', 'considering', 'buying']...)
2018-07-23 05:21:36,117 : INFO : adding document #20000 to Dictionary(156199 unique tokens: ['nuls', 'status', 'update', 'considering', 'buying']...)
2018-07-23 05:21:49,536 : INFO : built Dictionary(183094 unique tokens: ['nuls', 'status', 'update', 'considering', 'buying']...) from 24526 documents (total 3231880 corpus positions)


In [7]:
model = train_lda_model(100, dictionary, doc_term_matrix, 6, 10)

2018-07-23 05:23:06,890 : INFO : using symmetric alpha at 0.01
2018-07-23 05:23:06,896 : INFO : using symmetric eta at 5.461675423552929e-06
2018-07-23 05:23:06,922 : INFO : using serial LDA version on this node
2018-07-23 05:26:16,286 : INFO : running online (multi-pass) LDA training, 100 topics, 6 passes over the supplied corpus of 24526 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-07-23 05:26:16,287 : INFO : PROGRESS: pass 0, at document #2000/24526
2018-07-23 05:26:26,289 : INFO : merging changes from 2000 documents into a model of 24526 documents
2018-07-23 05:26:32,508 : INFO : topic #15 (0.010): 0.007*"btc" + 0.007*"bch" + 0.006*"fucking" + 0.006*"buy" + 0.006*"one" + 0.006*"coin" + 0.005*"bitcoin" + 0.005*"get" + 0.005*"like" + 0.005*"think"
2018-07-23 05:26:32,510 : INFO : topic #65 (0.010): 0.010*"get" + 0.008*"bitcoin" + 0.007*"money" + 0.006*"coin" + 0.006*"go" +

[(95,
  '0.180*"buy" + 0.085*"sell" + 0.024*"low" + 0.024*"price" + 0.023*"order" + '
  '0.021*"dip" + 0.020*"high" + 0.020*"wall" + 0.019*"buying" + '
  '0.017*"selling"'),
 (8,
  '0.077*"link" + 0.017*"chainlink" + 0.009*"token" + 0.009*"even" + '
  '0.009*"fucking" + 0.008*"swift" + 0.008*"oracle" + 0.008*"smart" + '
  '0.008*"know" + 0.007*"price"'),
 (63,
  '0.042*"silver" + 0.026*"amazon" + 0.017*"married" + 0.017*"factom" + '
  '0.014*"gold" + 0.014*"peter" + 0.012*"maximalist" + 0.011*"raiden" + '
  '0.011*"commodity" + 0.009*"illiterate"'),
 (19,
  '0.066*"future" + 0.056*"short" + 0.028*"leverage" + 0.022*"contract" + '
  '0.022*"margin" + 0.021*"price" + 0.019*"trading" + 0.018*"trade" + '
  '0.015*"position" + 0.014*"bitmex"'),
 (82,
  '0.033*"ha" + 0.011*"noice" + 0.011*"ti" + 0.010*"confirmed2000" + '
  '0.009*"believer" + 0.009*"breathe" + 0.009*"hah" + 0.008*"reeeeeeeeee" + '
  '0.008*"motion" + 0.007*"portal"'),
 (79,
  '0.157*"ark" + 0.016*"deluded" + 0.014*"staking" 

In [8]:
# model = LdaModel.load('models/ldamodel')

In [11]:
freqs = Counter()
get_highest_prob_topics = lambda x: x[1] > .1
first = itemgetter(0)
for thread in threads:
    tops = get_topics_for_document(thread, model, dictionary)
    topics = map(first, list(filter(get_highest_prob_topics, tops)))
    freqs[frozenset(topics)] += 1

In [13]:
second_element = itemgetter(1)
more_than_ten_documents = lambda x: x[1] > 10
more_than_one_topic = lambda x: len(x[0]) > 1
only_one_topic = lambda x: len(x[0]) == 1

key_topics = sorted(freqs.items(), key=second_element, reverse=True)
groups = list(filter(only_one_topic, key_topics))
groups = list(filter(more_than_ten_documents, groups))
pprint.pprint(groups)

[(frozenset({17}), 396),
 (frozenset({56}), 240),
 (frozenset({2}), 226),
 (frozenset({26}), 165),
 (frozenset({52}), 150),
 (frozenset({95}), 58),
 (frozenset({96}), 54),
 (frozenset({57}), 46),
 (frozenset({65}), 42),
 (frozenset({20}), 38),
 (frozenset({8}), 26),
 (frozenset({67}), 25),
 (frozenset({89}), 21),
 (frozenset({88}), 20),
 (frozenset({23}), 19),
 (frozenset({31}), 17),
 (frozenset({39}), 16),
 (frozenset({53}), 15),
 (frozenset({76}), 14),
 (frozenset({54}), 13),
 (frozenset({9}), 12),
 (frozenset({34}), 11),
 (frozenset({50}), 11),
 (frozenset({86}), 11),
 (frozenset({51}), 11),
 (frozenset({75}), 11)]


In [14]:
group_sets, _ = zip(*groups)
for grp in group_sets:
    print("="*30)    
    print(list(grp))
    for topic in zip(grp, map(model.show_topic, grp)):
        pprint.pprint(list(topic))
        print('\n')    

[17]
[17,
 [('btc', 0.029735321736004114),
  ('bch', 0.018235823450358846),
  ('going', 0.013776596735470992),
  ('fucking', 0.010818884225922205),
  ('bitcoin', 0.0095549130907119709),
  ('back', 0.0091081876417896043),
  ('go', 0.0085936161539116873),
  ('like', 0.0084320760866134926),
  ('fuck', 0.0078037559696356408),
  ('shit', 0.0075763207976318738)]]


[56]
[56,
 [('coin', 0.028034497569913053),
  ('moon', 0.016117897037794586),
  ('buy', 0.015017074082525554),
  ('im', 0.012590473691372782),
  ('going', 0.012421577701381407),
  ('get', 0.010448415553069707),
  ('like', 0.010434427390919336),
  ('good', 0.0094833308156218477),
  ('next', 0.0088402865556007817),
  ('eth', 0.0087181601285278854)]]


[2]
[2,
 [('get', 0.014059396868597262),
  ('like', 0.013046525369640703),
  ('make', 0.012511899133298872),
  ('money', 0.011190531505368638),
  ('people', 0.010223383373218478),
  ('im', 0.010194057903759305),
  ('want', 0.00861821638711429),
  ('know', 0.0081392533589371378),
  ('fu

In [ ]:
for topic in zip(topics, map(model.show_topic, topics)):
    print(topic)
    print('\n')

In [ ]:
model.top_topics(dictionary=dictionary, corpus=doc_term_matrix)